# Initialisation

In [1]:
from SL_model import SL_model
from spharm import legendre, GaussQuad
from time import time
import numpy as np
if __name__ == '__main__':
    model=SL_model(maxdeg=512,nb_workers=26,topo_it_max=50,k_max=50) # Generate the model base
    model.set_from_file('C:/Users/ahenry01/Desktop/Python_code/SL_C0de_output/Article Irrawaddy/vl/vl.VM5a.l50.um21.lm22.699result_27.09.2022_17.12.27')
    model.data_way='C:/Users/ahenry01/Desktop/Python_code/SL_C0de_data/'
    model.love_way=model.data_way+'variable_litosphere/VM5a.l50.um21.lm22.699'
    model.ice_way=model.data_way+'ice6G_data.mat'
    model.sed_way=model.data_way+'Irrawady_sedimentationgrid.mat'
    model.topo_way=model.data_way+'topo_SL.mat'
    model.P_lm=legendre(model.maxdeg,model.pool) #Calculate the Legendre associated functions for the Gaussian grid
    model.x, model.w= GaussQuad(model.maxdeg) # calculate the Gaussian grid parameters

    model.create_GRID() #create the grid framework

    model.create_parameters() #Create all grid parameters


    model.ice.quick_load(model.grid,model.pool,model.ice_way) #set the ice model
    model.sed.load(model,model.sed_way) # set the sedimentation model
    model.topo.load(model,model.topo_way) # set the topography model, this function need the sediment and ice model ! 
    #Disk_load=model.grid.disk(model,0,180,1,100)
    # we need to calculate the ocean function after setting the topography due to it's dependencies
    model.create_ocean(model.topo.topo_pres) # create the ocean function (see theoria (TABOO), spada et al., 2003) from the actual topography.
    model.sed.rho=model.data['rho_sed']
    model.oc.rho=model.data['rho_sea']
    model.ice.rho=model.data['rho_ice']
    model.grid.time_step=model.time_step
    model.ice.ice_corrected=model.data['ice_corrected']
    model.SL.delS.saved=model.data['delS']
    model.SL.sdelS.saved=model.data['sdelS']
    model.topo.topo=model.data['topo']

# Calcul de la déformation du géoïde et de la terre

## Ice Load

In [2]:
delL=np.zeros((model.time_step_number,int((model.maxdeg+1)*(model.maxdeg+2)/2)))+0j
sdelL=np.zeros((model.time_step_number,int((model.maxdeg+1)*(model.maxdeg+2)/2)))+0j
delL_prev=delL[0,:]
if not('beta_R_l' in model.love.__dict__):
    model.love.calc_beta_R(model)
if not('beta_G_l' in model.love.__dict__):
    model.love.calc_beta_G(model)
model.delR_e_i=np.zeros((model.time_step_number,int((model.maxdeg+1)*(model.maxdeg+2)/2)))+1j
model.delR_v_i=np.zeros((model.time_step_number,int((model.maxdeg+1)*(model.maxdeg+2)/2)))+1j
model.delG_e_i=np.zeros((model.time_step_number,int((model.maxdeg+1)*(model.maxdeg+2)/2)))+1j
model.delG_v_i=np.zeros((model.time_step_number,int((model.maxdeg+1)*(model.maxdeg+2)/2)))+1j
for t_it in range(1,model.time_step_number):
    delL[t_it,:]=model.ice.calc_del_ice(t_it).grdtocoeff(model).coeff*model.ice.rho
    sdelL[t_it-1,:]=delL[t_it,:]-delL[t_it-1,:]
    model.delR_e_i[t_it,:]=model.love.T.coeff*model.love.h.coeff*delL[t_it,:]
    model.load.calc_viscuous_load(model,t_it,sdelL,model.love.beta_R_l)
    model.delR_v_i[t_it,:]=model.love.T.coeff*model.load.V_lm.coeff
    model.delG_e_i[t_it,:]=model.love.T.coeff*model.love.k.coeff*delL[t_it,:]
    model.load.calc_viscuous_load(model,t_it,sdelL,model.love.beta_G_l)
    model.delG_v_i[t_it,:]=model.love.T.coeff*model.load.V_lm.coeff

## Sediment load

In [3]:
delL=np.zeros((model.time_step_number,int((model.maxdeg+1)*(model.maxdeg+2)/2)))+0j
sdelL=np.zeros((model.time_step_number,int((model.maxdeg+1)*(model.maxdeg+2)/2)))+0j
delL_prev=delL[0,:]
if not('beta_R_l' in model.love.__dict__):
    model.love.calc_beta_R(model)
if not('beta_G_l' in model.love.__dict__):
    model.love.calc_beta_G(model)
model.delR_e_s=np.zeros((model.time_step_number,int((model.maxdeg+1)*(model.maxdeg+2)/2)))+1j
model.delR_v_s=np.zeros((model.time_step_number,int((model.maxdeg+1)*(model.maxdeg+2)/2)))+1j
model.delG_e_s=np.zeros((model.time_step_number,int((model.maxdeg+1)*(model.maxdeg+2)/2)))+1j
model.delG_v_s=np.zeros((model.time_step_number,int((model.maxdeg+1)*(model.maxdeg+2)/2)))+1j
for t_it in range(1,model.time_step_number):
    delL[t_it,:]=model.sed.calc_del_sed(t_it).grdtocoeff(model).coeff*model.sed.rho
    sdelL[t_it-1,:]=delL[t_it,:]-delL[t_it-1,:]
    model.delR_e_s[t_it,:]=model.love.T.coeff*model.love.h.coeff*delL[t_it,:]
    model.load.calc_viscuous_load(model,t_it,sdelL,model.love.beta_R_l)
    model.delR_v_s[t_it,:]=model.love.T.coeff*model.load.V_lm.coeff
    model.delG_e_s[t_it,:]=model.love.T.coeff*model.love.k.coeff*delL[t_it,:]
    model.load.calc_viscuous_load(model,t_it,sdelL,model.love.beta_G_l)
    model.delG_v_s[t_it,:]=model.love.T.coeff*model.load.V_lm.coeff

## Ocean load

In [4]:
delL=np.zeros((model.time_step_number,int((model.maxdeg+1)*(model.maxdeg+2)/2)))+0j
sdelL=np.zeros((model.time_step_number,int((model.maxdeg+1)*(model.maxdeg+2)/2)))+0j
delL_prev=delL[0,:]
if not('beta_R_l' in model.love.__dict__):
    model.love.calc_beta_R(model)
if not('beta_G_l' in model.love.__dict__):
    model.love.calc_beta_G(model)
model.delR_e_o=np.zeros((model.time_step_number,int((model.maxdeg+1)*(model.maxdeg+2)/2)))+1j
model.delR_v_o=np.zeros((model.time_step_number,int((model.maxdeg+1)*(model.maxdeg+2)/2)))+1j
model.delG_e_o=np.zeros((model.time_step_number,int((model.maxdeg+1)*(model.maxdeg+2)/2)))+1j
model.delG_v_o=np.zeros((model.time_step_number,int((model.maxdeg+1)*(model.maxdeg+2)/2)))+1j
for t_it in range(1,model.time_step_number):
    print(t_it)
    delL[t_it,:]=model.SL.delS.saved[t_it,:]*model.oc.rho
    sdelL[t_it-1,:]=delL[t_it,:]-delL[t_it-1,:]
    model.delR_e_o[t_it,:]=model.love.T.coeff*model.love.h.coeff*delL[t_it,:]
    model.load.calc_viscuous_load(model,t_it,sdelL,model.love.beta_R_l)
    model.delR_v_o[t_it,:]=model.love.T.coeff*model.load.V_lm.coeff
    model.delG_e_o[t_it,:]=model.love.T.coeff*model.love.k.coeff*delL[t_it,:]
    model.load.calc_viscuous_load(model,t_it,sdelL,model.love.beta_G_l)
    model.delG_v_o[t_it,:]=model.love.T.coeff*model.load.V_lm.coeff

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46


# Relative Sea Level deformation

In [5]:
from spharm import sphericalobject
model.delSL=np.zeros((model.time_step_number,int((model.maxdeg+1)*(model.maxdeg+2)/2)))+0j
C=model.delG_v_o+model.delG_v_i+model.delG_v_s+model.delG_e_o+model.delG_e_i+model.delG_e_s-(model.delR_v_o+model.delR_v_i+model.delR_v_s+model.delR_e_o+model.delR_e_i+model.delR_e_s)
RSL_tot=sphericalobject(model.topo.topo[-1,:,:]-model.topo.topo[0,:,:]+model.ice.ice[0,:,:]-model.ice.ice[-1,:,:]).grdtocoeff(model)
model.topo.update_topo_0() 
model.oc.evaluate_ocean(model.topo.topo_0).grdtocoeff(model)
model.oc.update_oc_0()
for t_it in range(1,model.time_step_number) :
    print(model.time_step[t_it])
    model.sed.calc_del_sed(t_it).grdtocoeff(model)
    model.ice.calc_del_ice(t_it).grdtocoeff(model)
    delSLcurl_fl=sphericalobject(C[t_it,:],'coeff').coefftogrd(model)
    delSLcurl=sphericalobject(C[t_it,:]-model.sed.coeff-model.ice.coeff,'coeff').coefftogrd(model)
    model.oc.evaluate_ocean(model.topo.topo[t_it,:,:]).grdtocoeff(model)
    RO=sphericalobject(delSLcurl.grd*model.oc.grd).grdtocoeff(model)
    TO=sphericalobject(model.topo.topo_0*(model.oc.grd-model.oc.oc_0_grd)).grdtocoeff(model)
    #model.delPhi_g = np.real(1/model.oc.coeff[0] * (- model.ice.rho/model.oc.rho*model.ice.coeff[0] - RO.coeff[0] + TO.coeff[0]))
    model.delPhi_g = np.real(1/model.oc.coeff[0] * (- model.ice.rho/model.oc.rho*model.ice.coeff[0]))
    print(model.delPhi_g)
    model.delSL[t_it,:]=sphericalobject(delSLcurl_fl.grd+model.delPhi_g).grdtocoeff(model).coeff

25.0
5.012391600819793
24.0
9.041156919853089
23.0
11.427756054519962
22.0
13.759428240008454
21.0
15.263470483803346
20.5
16.166703794132395
20.0
17.027950534207317
19.5
18.00395174936999
19.0
18.888089972543735
18.5
19.93570375533739
18.0
21.45296538633115
17.5
23.191432619111488
17.0
24.80310325667243
16.5
26.56741140883974
16.0
28.62433448043218
15.5
32.64503823586035
15.0
36.10033409238351
14.5
39.973572488803356
14.0
61.59915147707139
13.5
68.06439140804295
13.0
72.47850074596707
12.5
78.38597677347667
12.0
82.48779893062391
11.5
87.53461226385954
11.0
97.52936641565357
10.5
102.37010050250058
10.0
107.32941608718076
9.5
111.93130177408888
9.0
115.53095619316412
8.5
119.46168454183416
8.0
122.4909048633507
7.5
125.12153734251028
7.0
126.52273415327568
6.5
127.40450864655597
6.0
127.72889719981846
5.5
127.79615480617383
5.0
127.87369879976498
4.5
127.94510388629996
4.0
128.0961888991766
3.5
128.116680858098
3.0
128.1322669026013
2.5
128.1920369239006
2.0
128.16798264979633
1.5
128

In [6]:
ESL=np.zeros((model.time_step_number,))
for i in range(model.time_step_number):
    ESL[i]=np.real(-model.ice.calc_del_ice(i).grdtocoeff(model).coeff[0]/model.oc.area*model.ice.rho/model.oc.rho)
ESL-=ESL.max()

# Loading and saving sata

In [ ]:
import pickle
way='C:/Users/ahenry01/Desktop/Python_code/SL_C0de_output/Article Irrawaddy/v_lm/v_lm.VM5a.l60.um21.lm22.449_postt'
with open(way+'.pkl', 'rb') as handle:
            data = pickle.load(handle)
model.delR_e_o=data['delR_e_o']
model.delR_e_s=data['delR_e_s']
model.delR_e_i=data['delR_e_i']
model.delR_v_o=data['delR_v_o']
model.delR_v_s=data['delR_v_s']
model.delR_v_i=data['delR_v_i']
model.delG_e_o=data['delG_e_o']
model.delG_e_s=data['delG_e_s']
model.delG_e_i=data['delG_e_i']
model.delG_v_o=data['delG_v_o']
model.delG_v_s=data['delG_v_s']
model.delG_v_i=data['delG_v_i']
#model.delSL=data['delSL']
#ESL=data['ESL']

In [1]:
import pickle
Data_dict={'delR_e_s':model.delR_e_s,'delR_v_s':model.delR_v_s,'delR_e_o':model.delR_e_o,'delR_v_o':model.delR_v_o,'delR_e_i':model.delR_e_i,'delR_v_i':model.delR_v_i,'delG_e_s':model.delG_e_s,'delG_v_s':model.delG_v_s,'delG_e_o':model.delG_e_o,'delG_v_o':model.delG_v_o,'delG_e_i':model.delG_e_i,'delG_v_i':model.delG_v_i,'delSL':model.delSL,'ESL':ESL}
output_way='C:/Users/ahenry01/Desktop/Python_code/SL_C0de_output/Article Irrawaddy/v_um/'
subname='vl.VM5a.l50.um21.lm22.699'
with open(output_way+subname+'_postt'+'.pkl', 'wb') as f:
            pickle.dump(Data_dict, f)

NameError: name 'model' is not defined

# Compute sequential deformation

## Sequential Ice load deformation

In [5]:
delL=np.zeros((model.time_step_number,int((model.maxdeg+1)*(model.maxdeg+2)/2)))+0j
sdelL=np.zeros((model.time_step_number,int((model.maxdeg+1)*(model.maxdeg+2)/2)))+0j
delL_prev=delL[0,:]
if not('beta_R_l' in model.love.__dict__):
    model.love.calc_beta_R(model)
if not('beta_G_l' in model.love.__dict__):
    model.love.calc_beta_G(model)
model.delR_e_i=np.zeros((model.time_step_number,int((model.maxdeg+1)*(model.maxdeg+2)/2)))+1j
model.delR_v_i=np.zeros((model.time_step_number,int((model.maxdeg+1)*(model.maxdeg+2)/2)))+1j
model.delG_e_i=np.zeros((model.time_step_number,int((model.maxdeg+1)*(model.maxdeg+2)/2)))+1j
model.delG_v_i=np.zeros((model.time_step_number,int((model.maxdeg+1)*(model.maxdeg+2)/2)))+1j
model.delR_v_decomp_i=np.zeros((model.time_step_number,model.time_step_number,int((model.maxdeg+1)*(model.maxdeg+2)/2)))+1j
model.delG_v_decomp_i=np.zeros((model.time_step_number,model.time_step_number,int((model.maxdeg+1)*(model.maxdeg+2)/2)))+1j
for t_it in range(1,model.time_step_number):
    delL[t_it,:]=model.ice.calc_del_ice(t_it).grdtocoeff(model).coeff*model.ice.rho
    sdelL[t_it-1,:]=delL[t_it,:]-delL[t_it-1,:]
    model.delR_e_i[t_it,:]=model.love.T.coeff*model.love.h.coeff*delL[t_it,:]
    model.load.calc_viscuous_load(model,t_it,sdelL,model.love.beta_R_l)
    model.delR_v_decomp_i[t_it-1,:t_it-1,:]=(model.love.beta_R_l[t_it-1,:t_it-1].transpose()*sdelL[:t_it-1].transpose()).transpose()
    model.delR_v_i[t_it,:]=model.love.T.coeff*model.load.V_lm.coeff
    model.delG_e_i[t_it,:]=model.love.T.coeff*model.love.k.coeff*delL[t_it,:]
    model.load.calc_viscuous_load(model,t_it,sdelL,model.love.beta_G_l)
    model.delG_v_decomp_i[t_it-1,:t_it-1,:]=(model.love.beta_G_l[t_it-1,:t_it-1].transpose()*sdelL[:t_it-1].transpose()).transpose()
    model.delG_v_i[t_it,:]=model.love.T.coeff*model.load.V_lm.coeff

## Sequencetial Sediment load deformation

In [6]:
delL=np.zeros((model.time_step_number,int((model.maxdeg+1)*(model.maxdeg+2)/2)))+0j
sdelL=np.zeros((model.time_step_number,int((model.maxdeg+1)*(model.maxdeg+2)/2)))+0j
delL_prev=delL[0,:]
if not('beta_R_l' in model.love.__dict__):
    model.love.calc_beta_R(model)
if not('beta_G_l' in model.love.__dict__):
    model.love.calc_beta_G(model)
model.delR_e_s=np.zeros((model.time_step_number,int((model.maxdeg+1)*(model.maxdeg+2)/2)))+1j
model.delR_v_s=np.zeros((model.time_step_number,int((model.maxdeg+1)*(model.maxdeg+2)/2)))+1j
model.delG_e_s=np.zeros((model.time_step_number,int((model.maxdeg+1)*(model.maxdeg+2)/2)))+1j
model.delG_v_s=np.zeros((model.time_step_number,int((model.maxdeg+1)*(model.maxdeg+2)/2)))+1j
model.delR_v_decomp_s=np.zeros((model.time_step_number,model.time_step_number,int((model.maxdeg+1)*(model.maxdeg+2)/2)))+1j
model.delG_v_decomp_s=np.zeros((model.time_step_number,model.time_step_number,int((model.maxdeg+1)*(model.maxdeg+2)/2)))+1j
for t_it in range(1,model.time_step_number):
    delL[t_it,:]=model.sed.calc_del_sed(t_it).grdtocoeff(model).coeff*model.sed.rho
    sdelL[t_it-1,:]=delL[t_it,:]-delL[t_it-1,:]
    model.delR_e_s[t_it,:]=model.love.T.coeff*model.love.h.coeff*delL[t_it,:]
    model.load.calc_viscuous_load(model,t_it,sdelL,model.love.beta_R_l)
    model.delR_v_decomp_s[t_it-1,:t_it-1,:]=(model.love.beta_R_l[t_it-1,:t_it-1].transpose()*sdelL[:t_it-1].transpose()).transpose()
    model.delR_v_s[t_it,:]=model.love.T.coeff*model.load.V_lm.coeff
    model.delG_e_s[t_it,:]=model.love.T.coeff*model.love.k.coeff*delL[t_it,:]
    model.load.calc_viscuous_load(model,t_it,sdelL,model.love.beta_G_l)
    model.delG_v_decomp_s[t_it-1,:t_it-1,:]=(model.love.beta_G_l[t_it-1,:t_it-1].transpose()*sdelL[:t_it-1].transpose()).transpose()
    model.delG_v_s[t_it,:]=model.love.T.coeff*model.load.V_lm.coeff

## Sequencetial Ocean load deformation

In [7]:
delL=np.zeros((model.time_step_number,int((model.maxdeg+1)*(model.maxdeg+2)/2)))+0j
sdelL=np.zeros((model.time_step_number,int((model.maxdeg+1)*(model.maxdeg+2)/2)))+0j
delL_prev=delL[0,:]
if not('beta_R_l' in model.love.__dict__):
    model.love.calc_beta_R(model)
if not('beta_G_l' in model.love.__dict__):
    model.love.calc_beta_G(model)
model.delR_e_o=np.zeros((model.time_step_number,int((model.maxdeg+1)*(model.maxdeg+2)/2)))+1j
model.delR_v_o=np.zeros((model.time_step_number,int((model.maxdeg+1)*(model.maxdeg+2)/2)))+1j
model.delG_e_o=np.zeros((model.time_step_number,int((model.maxdeg+1)*(model.maxdeg+2)/2)))+1j
model.delG_v_o=np.zeros((model.time_step_number,int((model.maxdeg+1)*(model.maxdeg+2)/2)))+1j
model.delR_v_decomp_o=np.zeros((model.time_step_number,model.time_step_number,int((model.maxdeg+1)*(model.maxdeg+2)/2)))+1j
model.delG_v_decomp_o=np.zeros((model.time_step_number,model.time_step_number,int((model.maxdeg+1)*(model.maxdeg+2)/2)))+1j
for t_it in range(1,model.time_step_number):
    print(t_it)
    delL[t_it,:]=model.SL.delS.saved[t_it,:]*model.oc.rho
    sdelL[t_it-1,:]=delL[t_it,:]-delL[t_it-1,:]
    model.delR_e_o[t_it,:]=model.love.T.coeff*model.love.h.coeff*delL[t_it,:]
    model.load.calc_viscuous_load(model,t_it,sdelL,model.love.beta_R_l)
    model.delR_v_decomp_o[t_it-1,:t_it-1,:]=(model.love.beta_R_l[t_it-1,:t_it-1].transpose()*sdelL[:t_it-1].transpose()).transpose()
    model.delR_v_o[t_it,:]=model.love.T.coeff*model.load.V_lm.coeff
    model.delG_e_o[t_it,:]=model.love.T.coeff*model.love.k.coeff*delL[t_it,:]
    model.load.calc_viscuous_load(model,t_it,sdelL,model.love.beta_G_l)
    model.delG_v_decomp_o[t_it-1,:t_it-1,:]=(model.love.beta_G_l[t_it-1,:t_it-1].transpose()*sdelL[:t_it-1].transpose()).transpose()
    model.delG_v_o[t_it,:]=model.love.T.coeff*model.load.V_lm.coeff

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46


## Saving code

In [10]:
import pickle
import joblib

Data_dict={'delR_e_s':model.delR_e_s,'delR_v_s':model.delR_v_s,'delR_e_o':model.delR_e_o,'delR_v_o':model.delR_v_o,'delR_e_i':model.delR_e_i,'delR_v_i':model.delR_v_i,'delG_e_s':model.delG_e_s,'delG_v_s':model.delG_v_s,'delG_e_o':model.delG_e_o,'delG_v_o':model.delG_v_o,'delG_e_i':model.delG_e_i,'delG_v_i':model.delG_v_i,'delSL':model.delSL,'ESL':ESL,'delR_v_decomp_i':model.delR_v_decomp_i,'delG_v_decomp_i':model.delG_v_decomp_i,'delR_v_decomp_s':model.delR_v_decomp_s,'delG_v_decomp_s':model.delG_v_decomp_s,'delR_v_decomp_i':model.delR_v_decomp_o,'delG_v_decomp_i':model.delG_v_decomp_o}
output_way='C:/Users/ahenry01/Desktop/Python_code/SL_C0de_output/Article Irrawaddy/'
subname='base.VM5a.l60.um21.lm22.699'
with open(output_way+subname+'_postt'+'.pkl', 'wb') as f:
            joblib.dump(Data_dict, f)